## Entregable 1
#(Use el ejemplo de Lucas para poder hacer el entregable ya que de ninguna manera podia ahcer la conexión al redshift)

Pasos a seguir:
* Bajar datos de una API en formato JSON
* Cargar datos en la tabla de Redshift

Esto lo vamos a llevar a cabo usando `requests`, `Spark` y un driver de conexión de `Postgres`

## 1) Bajar datos de una API en formato JSON
Para este ejemplo vamos a usar la API de [Datos Argentina](https://www.datos.gob.ar/)

Y nos vamos a traer los datos de: Exportaciones de cereales. En millones de dólares FOB

Para probar la API ir a: [API de Series de Tiempo AR: Generador de URLs](https://datosgobar.github.io/series-tiempo-ar-call-generator/)

In [1]:
import requests
import urllib.parse

def get_api_call(ids, **kwargs):
    API_BASE_URL = "https://apis.datos.gob.ar/series/api/"
    kwargs["ids"] = ",".join(ids)
    return "{}{}?{}".format(API_BASE_URL, "series", urllib.parse.urlencode(kwargs))

In [2]:
# Ejemplo: https://apis.datos.gob.ar/series/api/series?ids=Automotriz_expos_ItCfsr
api_call = get_api_call(["Automotriz_expos_ItCfsr"])
print(api_call)

https://apis.datos.gob.ar/series/api/series?ids=Automotriz_expos_ItCfsr


In [3]:
result = requests.get(api_call).json()
print(result)

{'data': [['2003-01-01', 9504.0], ['2003-02-01', 7216.0], ['2003-03-01', 9454.0], ['2003-04-01', 10656.0], ['2003-05-01', 8671.0], ['2003-06-01', 8715.0], ['2003-07-01', 8633.0], ['2003-08-01', 9012.0], ['2003-09-01', 9999.0], ['2003-10-01', 8838.0], ['2003-11-01', 7646.0], ['2003-12-01', 9713.0], ['2004-01-01', 5323.0], ['2004-02-01', 10250.0], ['2004-03-01', 10857.0], ['2004-04-01', 11559.0], ['2004-05-01', 11539.0], ['2004-06-01', 8567.0], ['2004-07-01', 13011.0], ['2004-08-01', 12648.0], ['2004-09-01', 14900.0], ['2004-10-01', 16473.0], ['2004-11-01', 16448.0], ['2004-12-01', 14661.0], ['2005-01-01', 10804.0], ['2005-02-01', 9007.0], ['2005-03-01', 15134.0], ['2005-04-01', 14291.0], ['2005-05-01', 14137.0], ['2005-06-01', 13177.0], ['2005-07-01', 16753.0], ['2005-08-01', 18019.0], ['2005-09-01', 15995.0], ['2005-10-01', 19783.0], ['2005-11-01', 19671.0], ['2005-12-01', 14810.0], ['2006-01-01', 5155.0], ['2006-02-01', 12793.0], ['2006-03-01', 19557.0], ['2006-04-01', 19492.0], ['200

## 2) Cargar datos en la tabla de Redshift

```SQL
create table if not exists fabiolecce93_coderhouse.vehiculos_exportados (
    date_from VARCHAR(10) distkey,
    cantidad decimal(10),
    frequency varchar(12)
) sortkey(date_from);
```

(Aca un tema que no entendi por que me pasaba, es que intente colocar la columna "cantidad" como INT y me tiraba error)

In [4]:
!pip install psycopg2-binary

In [5]:
# Crear sesion de Spark
import os
import psycopg2

from pyspark.sql import SparkSession
from pyspark.sql.functions import when, lit, col

# Postgres and Redshift JDBCs
driver_path = "/home/coder/working_dir/driver_jdbc/postgresql-42.2.27.jre7.jar"

os.environ['PYSPARK_SUBMIT_ARGS'] = f'--driver-class-path {driver_path} --jars {driver_path} pyspark-shell'
os.environ['SPARK_CLASSPATH'] = driver_path

# Create SparkSession 
spark = SparkSession.builder \
        .master("local") \
        .appName("Conexion entre Pyspark y Redshift") \
        .config("spark.jars", driver_path) \
        .config("spark.executor.extraClassPath", driver_path) \
        .getOrCreate()

In [6]:
env = os.environ

Revisar documentación:
* [AWS Redshift + Spark documentation](https://docs.aws.amazon.com/emr/latest/ReleaseGuide/emr-spark-redshift.html)
* [Spark + Redshift connector](https://github.com/spark-redshift-community/spark-redshift#readme)

In [7]:
# Connect to Redshift using psycopg2
conn = psycopg2.connect(
    host=env['AWS_REDSHIFT_HOST'],
    port=env['AWS_REDSHIFT_PORT'],
    dbname=env['AWS_REDSHIFT_DBNAME'],
    user=env['AWS_REDSHIFT_USER'],
    password=env['AWS_REDSHIFT_PASSWORD']
)

In [8]:
cursor = conn.cursor()
cursor.execute(f"""
create table if not exists {env['AWS_REDSHIFT_SCHEMA']}.vehiculos_exportados (
    date_from VARCHAR(10) distkey,
    cantidad decimal(10),
    frequency varchar(12)
) sortkey(date_from);
""")
conn.commit()
cursor.close()
print("Table created!")

Table created!


In [9]:
cursor = conn.cursor()
cursor.execute(f"""
SELECT
  distinct tablename
FROM
  PG_TABLE_DEF
WHERE
  schemaname = '{env['AWS_REDSHIFT_SCHEMA']}';
""")
# resultado = cursor.fetchall()
print(", ".join(map(lambda x: x[0], cursor.fetchall())))
cursor.close()

vehiculos_exportados


In [10]:
# Create the DataFrame with the specified column names
df = spark.createDataFrame(result['data'], ["date_from", "cantidad"])

In [11]:
df.printSchema()
df.show()

root
 |-- date_from: string (nullable = true)
 |-- cantidad: double (nullable = true)

+----------+--------+
| date_from|cantidad|
+----------+--------+
|2003-01-01|  9504.0|
|2003-02-01|  7216.0|
|2003-03-01|  9454.0|
|2003-04-01| 10656.0|
|2003-05-01|  8671.0|
|2003-06-01|  8715.0|
|2003-07-01|  8633.0|
|2003-08-01|  9012.0|
|2003-09-01|  9999.0|
|2003-10-01|  8838.0|
|2003-11-01|  7646.0|
|2003-12-01|  9713.0|
|2004-01-01|  5323.0|
|2004-02-01| 10250.0|
|2004-03-01| 10857.0|
|2004-04-01| 11559.0|
|2004-05-01| 11539.0|
|2004-06-01|  8567.0|
|2004-07-01| 13011.0|
|2004-08-01| 12648.0|
+----------+--------+
only showing top 20 rows



In [12]:
df_to_write = df.withColumn('frequency', lit('Mensual'))
df_to_write.printSchema()
df_to_write.show()

root
 |-- date_from: string (nullable = true)
 |-- cantidad: double (nullable = true)
 |-- frequency: string (nullable = false)

+----------+--------+---------+
| date_from|cantidad|frequency|
+----------+--------+---------+
|2003-01-01|  9504.0|  Mensual|
|2003-02-01|  7216.0|  Mensual|
|2003-03-01|  9454.0|  Mensual|
|2003-04-01| 10656.0|  Mensual|
|2003-05-01|  8671.0|  Mensual|
|2003-06-01|  8715.0|  Mensual|
|2003-07-01|  8633.0|  Mensual|
|2003-08-01|  9012.0|  Mensual|
|2003-09-01|  9999.0|  Mensual|
|2003-10-01|  8838.0|  Mensual|
|2003-11-01|  7646.0|  Mensual|
|2003-12-01|  9713.0|  Mensual|
|2004-01-01|  5323.0|  Mensual|
|2004-02-01| 10250.0|  Mensual|
|2004-03-01| 10857.0|  Mensual|
|2004-04-01| 11559.0|  Mensual|
|2004-05-01| 11539.0|  Mensual|
|2004-06-01|  8567.0|  Mensual|
|2004-07-01| 13011.0|  Mensual|
|2004-08-01| 12648.0|  Mensual|
+----------+--------+---------+
only showing top 20 rows



In [13]:
df_to_write.write \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{env['AWS_REDSHIFT_HOST']}:{env['AWS_REDSHIFT_PORT']}/{env['AWS_REDSHIFT_DBNAME']}") \
    .option("dbtable", f"{env['AWS_REDSHIFT_SCHEMA']}.vehiculos_exportados") \
    .option("user", env['AWS_REDSHIFT_USER']) \
    .option("password", env['AWS_REDSHIFT_PASSWORD']) \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

In [14]:
# Query Redshift using Spark SQL
query = f"select * from {env['AWS_REDSHIFT_SCHEMA']}.vehiculos_exportados"
data = spark.read \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{env['AWS_REDSHIFT_HOST']}:{env['AWS_REDSHIFT_PORT']}/{env['AWS_REDSHIFT_DBNAME']}") \
    .option("dbtable", f"({query}) as tmp_table") \
    .option("user", env['AWS_REDSHIFT_USER']) \
    .option("password", env['AWS_REDSHIFT_PASSWORD']) \
    .option("driver", "org.postgresql.Driver") \
    .load()

In [15]:
data.printSchema()
data.show()

root
 |-- date_from: string (nullable = true)
 |-- cantidad: double (nullable = true)
 |-- frequency: string (nullable = true)

+----------+--------+---------+
| date_from|cantidad|frequency|
+----------+--------+---------+
|2003-01-01|  9504.0|  Mensual|
|2003-02-01|  7216.0|  Mensual|
|2003-03-01|  9454.0|  Mensual|
|2003-04-01| 10656.0|  Mensual|
|2003-05-01|  8671.0|  Mensual|
|2003-06-01|  8715.0|  Mensual|
|2003-07-01|  8633.0|  Mensual|
|2003-08-01|  9012.0|  Mensual|
|2003-09-01|  9999.0|  Mensual|
|2003-10-01|  8838.0|  Mensual|
|2003-11-01|  7646.0|  Mensual|
|2003-12-01|  9713.0|  Mensual|
|2004-01-01|  5323.0|  Mensual|
|2004-02-01| 10250.0|  Mensual|
|2004-03-01| 10857.0|  Mensual|
|2004-04-01| 11559.0|  Mensual|
|2004-05-01| 11539.0|  Mensual|
|2004-06-01|  8567.0|  Mensual|
|2004-07-01| 13011.0|  Mensual|
|2004-08-01| 12648.0|  Mensual|
+----------+--------+---------+
only showing top 20 rows



In [16]:
conn.close()